#### Load Data

In [ ]:
import pandas as pd

def load_data():
  df_cbe = pd.read_csv('/content/cbe_cleaned.csv')
  df_boa = pd.read_csv('/content/boa_cleaned.csv')
  df_dashen = pd.read_csv('/content/dashen_cleaned.csv')
  return df_cbe, df_boa, df_dashen

df_cbe, df_boa, df_dashen = load_data()

In [ ]:
df_cbe.head()

,review,rating,date,bank,source
0,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,cbe,playstore
1,what is this app problem???,1,2025-06-05,cbe,playstore
2,the app is proactive and a good connections.,5,2025-06-05,cbe,playstore
3,I cannot send to cbebirr app. through this app.,3,2025-06-05,cbe,playstore
4,good,4,2025-06-05,cbe,playstore


In [ ]:
df_boa.head()

,review,rating,date,bank,source
0,it's not working,3,2025-06-05,boa,playstore
1,"Hello, I’m facing a problem with the BOA Mobil...",1,2025-06-03,boa,playstore
2,exceptional,5,2025-06-03,boa,playstore
3,BoA Mobile good bank,5,2025-06-02,boa,playstore
4,this is worest app 24/7 loading,1,2025-06-01,boa,playstore


In [ ]:
df_dashen.head()

,review,rating,date,bank,source
0,love,3,2025-06-06,dashen,playstore
1,መቸሸጠ,5,2025-06-03,dashen,playstore
2,wow,5,2025-06-03,dashen,playstore
3,gadaa,5,2025-06-01,dashen,playstore
4,Massive upgrade from the Amole app.,5,2025-05-31,dashen,playstore


In [15]:
df_cbe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7498 entries, 0 to 7497
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  7491 non-null   object
 1   rating  7498 non-null   int64 
 2   date    7498 non-null   object
 3   bank    7498 non-null   object
 4   source  7498 non-null   object
dtypes: int64(1), object(4)
memory usage: 293.0+ KB


#### Ensure all reviews are strings

In [20]:
df_cbe['review'] = df_cbe['review'].astype(str)
df_dashen['review'] = df_dashen['review'].astype(str)
df_boa['review'] = df_boa['review'].astype(str)

In [17]:
df_cbe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7498 entries, 0 to 7497
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  7498 non-null   object
 1   rating  7498 non-null   int64 
 2   date    7498 non-null   object
 3   bank    7498 non-null   object
 4   source  7498 non-null   object
dtypes: int64(1), object(4)
memory usage: 293.0+ KB


#### Apply sentiment analysis

In [ ]:
from transformers import pipeline

classifier = pipeline(task = 'sentiment-analysis',
                      model = 'distilbert-base-uncased-finetuned-sst-2-english')

In [18]:
# Calculate sentiment scores and remarks for playstore reviews
# CBE
df_cbe[['sentiment', 'sentiment_score']] = df_cbe['review'].apply(  # noqa
  lambda x: pd.Series(classifier(x)[0]))

In [21]:
# Calculate sentiment scores and remarks for playstore reviews
# Dashen
df_dashen[['sentiment', 'sentiment_score']] = df_dashen['review'].apply(  # noqa
    lambda x: pd.Series(classifier(x)[0]))

In [22]:
# Calculate sentiment scores and remarks for playstore reviews
# CBE
df_boa[['sentiment', 'sentiment_score']] = df_boa['review'].apply(  # noqa
  lambda x: pd.Series(classifier(x)[0]))

In [ ]:
df_cbe.to_csv('cbe_sentiment.csv')
df_boa.to_csv('boa_sentiment.csv')
df_dashen.to_csv('dashen_sentiment.csv')

#### Replace app names with their playstore app names

In [29]:
df_cbe['bank'] = 'Commercial Bank of Ethiopia'
df_boa['bank'] = 'BoA Mobile'
df_dashen['bank'] = 'Dashen Bank'

In [27]:
df_cbe.head()

,review,rating,date,bank,source,sentiment,sentiment_score
0,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,Commercial Bank of Ethiopia,playstore,NEGATIVE,0.996465
1,what is this app problem???,1,2025-06-05,Commercial Bank of Ethiopia,playstore,NEGATIVE,0.999623
2,the app is proactive and a good connections.,5,2025-06-05,Commercial Bank of Ethiopia,playstore,POSITIVE,0.999868
3,I cannot send to cbebirr app. through this app.,3,2025-06-05,Commercial Bank of Ethiopia,playstore,NEGATIVE,0.995335
4,good,4,2025-06-05,Commercial Bank of Ethiopia,playstore,POSITIVE,0.999816


In [12]:
df_dashen.head()

,review,rating,date,bank,source,sentiment,sentiment_score
0,love,3,2025-06-06,dashen,playstore,POSITIVE,0.999874
1,መቸሸጠ,5,2025-06-03,dashen,playstore,NEGATIVE,0.697057
2,wow,5,2025-06-03,dashen,playstore,POSITIVE,0.999592
3,gadaa,5,2025-06-01,dashen,playstore,NEGATIVE,0.721146
4,Massive upgrade from the Amole app.,5,2025-05-31,dashen,playstore,POSITIVE,0.964642


In [38]:
def missing_values_per_column(df):
    return df[['sentiment', 'sentiment_score']].isnull().sum()

def calculate_missing_values_per_column():
    print("------------------------------------")
    print("CBE BANK NO OF MISSING VALUES IN SENTIMENT AND SENTIMENT SCORE COLUMNS")
    result = missing_values_per_column(df_cbe)
    print(f"{result}")
    print("------------------------------------")
    print("BOA BANK NO OF MISSING VALUES IN SENTIMENT AND SENTIMENT SCORE COLUMNS")
    result = missing_values_per_column(df_boa)
    print(f"{result}")
    print("------------------------------------")
    print("DASHEN BANK  NO OF MISSING VALUES IN SENTIMENT AND SENTIMENT SCORE COLUMNS")
    result = missing_values_per_column(df_dashen)
    print(f"{result}")

calculate_missing_values_per_column()

------------------------------------
CBE BANK NO OF MISSING VALUES IN SENTIMENT AND SENTIMENT SCORE COLUMNS
sentiment          0
sentiment_score    0
dtype: int64
------------------------------------
BOA BANK NO OF MISSING VALUES IN SENTIMENT AND SENTIMENT SCORE COLUMNS
sentiment          0
sentiment_score    0
dtype: int64
------------------------------------
DASHEN BANK  NO OF MISSING VALUES IN SENTIMENT AND SENTIMENT SCORE COLUMNS
sentiment          0
sentiment_score    0
dtype: int64


From the above output we can see that their are no mising values in columns "sentiment" and "sentiment score" hence we can conclude that sentiment analysis was done successfully for all rows.

#### Aggregate by Bank Name and rating

In [56]:
def aggregate():
  print("CBE AGGREGATE")
  print(df_cbe.groupby(['bank', 'rating', 'sentiment']).agg({'sentiment': ['count']}))
  print("BOA AGGREGATE")
  print(df_boa.groupby(['bank', 'rating', 'sentiment']).agg({'sentiment': ['count']}))
  print("DASHEN AGGREGATE")
  print(df_dashen.groupby(['bank', 'rating', 'sentiment']).agg({'sentiment': ['count']}))
aggregate()

CBE AGGREGATE
                                             sentiment
                                                 count
bank                        rating sentiment          
Commercial Bank of Ethiopia 1      NEGATIVE        925
                                   POSITIVE        197
                            2      NEGATIVE        233
                                   POSITIVE         67
                            3      NEGATIVE        286
                                   POSITIVE        184
                            4      NEGATIVE        270
                                   POSITIVE        501
                            5      NEGATIVE        822
                                   POSITIVE       4013
BOA AGGREGATE
                            sentiment
                                count
bank       rating sentiment          
BoA Mobile 1      NEGATIVE        382
                  POSITIVE         40
           2      NEGATIVE         46
                  POSITIVE   